In [1]:
import numpy as np

LIMIT = 10
class Dataset:
    def __init__(self, LIMIT=10, dir_path="drive/MyDrive/Datasets"):
        self.dataset = [[[] for _ in range(10)] for _ in range(10)]
        self.data = []
        self.labels = []
        limit_the_search = [j for i in range(10) for k in range(10) for j in range(i*100+(k*10)+1, i*100+(k*10)+LIMIT+1)]
        for i in limit_the_search:
          filepath = "{}/{}.txt".format(dir_path, i)
          try:
              lines = [line.strip() for line in open(filepath, "r")]
              class_id =  int(lines[1][len(lines[1]) - 1])
              user_id = int(lines[2][len(lines[2]) - 1])
              user_data = []
              for i, line in enumerate(lines[5:]):
                  if line:
                      user_data.append(list(map(float, line.split(',')))[0:3])
              self.dataset[user_id - 1][class_id - 1].append(user_data) 
              self.data.append(user_data)
              self.labels.append(class_id)
          except IOError as e:
              print("Unable to read dataset file {}!\n".format(filepath))

    def get_user(self, index):
        """Returns an array containing all the data saved for the user <<index>>"""
        return len(self.data[index])

dataset = Dataset()

In [2]:
# import tslearn
import sys
!{sys.executable} -m pip install tslearn

In [ ]:
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.svm import TimeSeriesSVC
from tslearn.utils import to_time_series_dataset
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

np.random.seed(0)
LIMIT= LIMIT*10

x = np.array(dataset.data)
labels = np.array(dataset.labels)

accuracies = []
for user_id in range(10):
  # split the dataset
  indexes = range(user_id*LIMIT, user_id*LIMIT+LIMIT)
  #print(list(indexes))
  train_set = to_time_series_dataset(np.delete(x, indexes))
  train_labels = np.delete(labels, indexes)
  test_labels = labels[indexes]
  test_set = to_time_series_dataset(x[indexes])

  x_train = TimeSeriesScalerMinMax().fit_transform(train_set)
  x_test = TimeSeriesScalerMinMax().fit_transform(test_set)

  model = TimeSeriesSVC(kernel="gak", gamma="auto", probability=True)
  # Prediction
  model.fit(x_train, train_labels)
  accuracy = model.score(x_test, test_labels)
  print("The accuracy pour l'utilisateur {}: {}".format(user_id+1, accuracy))
  accuracies.append(accuracy)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
